In [1]:
from pyspark.sql import SparkSession
from pyspark.ml.linalg import VectorUDT, DenseVector
from pyspark.sql.functions import udf
from pyspark.sql.types import ArrayType, DoubleType
from functools import reduce

In [2]:
spark = SparkSession.builder.appName("Prediction to ElasticResearch").master("local[*]").getOrCreate()
spark.sparkContext.setLogLevel("ERROR")

:: loading settings :: url = jar:file:/opt/spark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /root/.ivy2/cache
The jars for the packages stored in: /root/.ivy2/jars
org.apache.spark#spark-sql-kafka-0-10_2.12 added as a dependency
org.apache.spark#spark-avro_2.12 added as a dependency
org.elasticsearch#elasticsearch-spark-30_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-af4fa8cd-ac1a-42c7-8467-4cf93e1bca22;1.0
	confs: [default]
	found org.apache.spark#spark-sql-kafka-0-10_2.12;3.5.0 in central
	found org.apache.spark#spark-token-provider-kafka-0-10_2.12;3.5.0 in central
	found org.apache.kafka#kafka-clients;3.4.1 in central
	found org.lz4#lz4-java;1.8.0 in central
	found org.xerial.snappy#snappy-java;1.1.10.3 in central
	found org.slf4j#slf4j-api;2.0.7 in central
	found org.apache.hadoop#hadoop-client-runtime;3.3.4 in central
	found org.apache.hadoop#hadoop-client-api;3.3.4 in central
	found commons-logging#commons-logging;1.1.3 in central
	found com.google.code.findbugs#jsr305;3.0.0 in central
	found org.

In [4]:
import subprocess

cmd = ["hdfs","dfs","-ls","-R","/workspace/output/prediction/svm_pred"]
result = subprocess.run(cmd,capture_output=True,text=True)
lines = result.stdout.strip().split("\n")
files = []
for line in lines:
    parts = line.split()
    if len(parts) == 8:
        filepath = parts[-1]
        filename = filepath.split("/")[-1]
        if filename.startswith("comment"):
           files.append(filepath)

def vector_to_array(v):
    if v is None:
        return None
    return v.toArray().tolist()

vector_to_array_udf = udf(vector_to_array, ArrayType(DoubleType()))

for file in files:
    df = spark.read.parquet(f"hdfs://namenode:9000{file}")
    df2 = df \
        .withColumn("features_array", vector_to_array_udf("features")) \
        .withColumn("rawPrediction_array", vector_to_array_udf("rawPrediction")) \
        .drop("features","rawPrediction")
    df_elastic = df2.select("text", "features_array", "prediction", "rawPrediction_array")
    df_elastic.write \
      .format("org.elasticsearch.spark.sql") \
      .option("es.nodes", "elasticsearch") \
      .option("es.port", "9200") \
      .option("es.nodes.wan.only", "true") \
      .mode("append") \
      .save("svm_pred")

In [5]:
spark.stop()